# Imports

In [1]:
import pandas as pd
import numpy as np

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
from statsmodels.tsa.arima_model import ARMA

In [3]:
from sklearn.metrics import mean_squared_error as mse

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as plt

## Clean data

In [6]:
def clean_data():
    
    global db1
    
    # Number of Infections greater than 1:
    db1 = db1[(db1.num_infections > 0)].reset_index()
    db1 = db1.drop(columns = ['index'], axis = 1)
    
    # Sort ascending order by date:
    db1['date'] = pd.to_datetime(db1.date)
    db1 = db1.sort_values(by="date").reset_index()
    db1 = db1.drop(columns = ['index'], axis = 1)
   
    # Data cleaning autonomous_region:
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Valenciana' if x == 'Valenciana, Comunidad' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad de Madrid' if x == 'Madrid, Comunidad de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Región de de Murcia' if x == 'Murcia, Región de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Foral de Navarra' if x == 'Navarra, Comunidad Foral de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Principado de Asturias' if x == 'Asturias, Principado de' else x)
    
    # Data cleaning province:
    db1['province'] = db1.province.apply(lambda x: 'Alicante' if x == 'Alicante/Alacant' else x)
    db1['province'] = db1.province.apply(lambda x: 'Castellón' if x == 'Castellón/Castelló' else x)
    db1['province'] = db1.province.apply(lambda x: 'Araba' if x == 'Araba/Álava' else x)
    db1['province'] = db1.province.apply(lambda x: 'Valencia' if x == 'Valencia/València' else x)
    
    # Data cleaning sex and age_interval:
    db1 = db1[(db1.sex != 'NC') & (db1.age_interval != 'NC')]

In [7]:
def cumulative(dataframe):
    
    cumu_num_infections = dataframe.num_infections.cumsum()
    cumu_num_hosp = dataframe.num_hosp.cumsum()
    cumu_num_uci = dataframe.num_uci.cumsum()
    cumu_num_dead = dataframe.num_dead.cumsum()

    cumulative = pd.DataFrame({'cumu_num_infections': cumu_num_infections, 'cumu_num_hosp': cumu_num_hosp, 
                               'cumu_num_uci': cumu_num_uci, 'cumu_num_dead': cumu_num_dead})
    dataframe = pd.concat([dataframe, cumulative], axis = 1)
    return dataframe

In [8]:
def mov_7_ave(dataframe):
    dataframe['ave_7_num_infections'] = dataframe.iloc[:,1].rolling(window=7).mean()
    dataframe['ave_7_num_hosp'] = dataframe.iloc[:,2].rolling(window=7).mean()
    dataframe['ave_7_num_uci'] = dataframe.iloc[:,3].rolling(window=7).mean()
    dataframe['ave_7_num_dead'] = dataframe.iloc[:,4].rolling(window=7).mean()
    
    return dataframe

In [9]:
def evolution(dataframe):
    
    
    bydate = dataframe.groupby('date').sum().reset_index()
    bydate = cumulative(bydate)
    bydate = mov_7_ave(bydate)
    
    return bydate

In [10]:
def organise(column):
    
    global db1

    name = db1.groupby(column).sum()
    freq_rel(name)
    
    return name

In [11]:
def freq_rel(dataframe):
    
    dataframe['Number of infection (%)'] = [round(i/ dataframe['num_infections'].sum(),3)*100 for i in dataframe['num_infections']]
    dataframe['Number of hospitalisation (%)'] = [round(i/ dataframe['num_hosp'].sum(),3)*100 for i in dataframe['num_hosp']]
    dataframe['Number of Intensive Care Unit (%)'] = [round(i/ dataframe['num_uci'].sum(),3)*100 for i in dataframe['num_uci']]
    dataframe['Number of deaths (%)'] = [round(i/ dataframe['num_dead'].sum(),3)*100 for i in dataframe['num_dead']]
    
    return dataframe

In [15]:
db1 = pd.read_csv('covid-19-sample.csv')

In [17]:
clean_data()

In [18]:
db1

,date,autonomous_region,province,sex,age_interval,num_infections,num_hosp,num_uci,num_dead
0,2020-01-18,Comunidad de Madrid,Madrid,M,40-49,1,1,0,0
1,2020-01-31,Canarias,Santa Cruz de Tenerife,H,20-29,1,0,0,0
2,2020-02-09,Illes Balears,Illes Balears,H,40-49,1,1,0,0
3,2020-02-12,Catalunya,Barcelona,M,30-39,1,0,0,0
4,2020-02-12,Comunidad de Madrid,Madrid,M,50-59,1,0,0,0
...,...,...,...,...,...,...,...,...,...
300931,2021-06-10,Andalucía,Huelva,M,80+,1,0,0,0
300932,2021-06-10,Andalucía,Huelva,M,60-69,2,0,0,0
300933,2021-06-10,Andalucía,Huelva,M,50-59,5,0,0,0
300934,2021-06-10,Andalucía,Córdoba,H,80+,2,0,0,0


In [64]:
bydate=organise('date')

In [65]:
bydate['ave_7_num_infections'] = bydate.iloc[:,0].rolling(window=7).mean()

In [66]:
infections = bydate.drop(columns=['num_infections', 'num_hosp', 'num_uci', 'num_dead', 'Number of infection (%)',
                              'Number of hospitalisation (%)', 'Number of Intensive Care Unit (%)', 'Number of deaths (%)'], axis = 1)

# Infections

In [67]:
infections = infections[6:]

In [70]:
infections = infections.append(pd.Series(name='2021-06-11 00:00:00'))
infections = infections.append(pd.Series(name='2021-06-12 00:00:00'))
infections = infections.append(pd.Series(name='2021-06-13 00:00:00'))
infections = infections.append(pd.Series(name='2021-06-14 00:00:00'))
infections = infections.append(pd.Series(name='2021-06-15 00:00:00'))
infections = infections.append(pd.Series(name='2021-06-16 00:00:00'))
infections = infections.append(pd.Series(name='2021-06-17 00:00:00'))

,ave_7_num_infections
date,
2021-06-09 00:00:00,3582.571429
2021-06-10 00:00:00,3493.571429
2021-06-11 00:00:00,NaN
2021-06-11 00:00:00,NaN
2021-06-12 00:00:00,NaN
2021-06-13 00:00:00,NaN
2021-06-14 00:00:00,NaN
2021-06-15 00:00:00,NaN
2021-06-16 00:00:00,NaN


In [30]:
train, test = infections[:-7], infections[-7:]

In [34]:
model=ARMA(train, order = (19, 7)).fit(disp=False)
print('rsme:', mse(model.predict(len(train), len(infections)-1), test)**0.5, 'daily infections')

rsme: 82.1678386393958 daily infections


In [78]:
train, test = infections[:-8], infections[-8:]

In [79]:
train

,ave_7_num_infections
date,
2020-02-23,1.428571
2020-02-24,1.571429
2020-02-25,1.571429
2020-02-26,2.428571
2020-02-27,5.285714
...,...
2021-06-06,4004.000000
2021-06-07,3899.857143
2021-06-08,3749.428571


In [80]:
model = ARMA(train, order = (19, 7)).fit(disp=False)
pred = model.predict(len(train), len(infections)-1)

In [82]:
pred

2021-06-11    3430.818823
2021-06-12    3353.771666
2021-06-13    3191.312601
2021-06-14    3046.618273
2021-06-15    2952.195890
2021-06-16    2946.160710
2021-06-17    3008.848439
2021-06-18    3093.853175
Freq: D, dtype: float64

In [83]:
prediction = pd.DataFrame(pred, columns = ['pred_num_infections'])

In [84]:
prediction

,pred_num_infections
2021-06-11,3430.818823
2021-06-12,3353.771666
2021-06-13,3191.312601
2021-06-14,3046.618273
2021-06-15,2952.195890
2021-06-16,2946.160710
2021-06-17,3008.848439
2021-06-18,3093.853175


## Hospitalisations

In [110]:
hosp = bydate.drop(columns=['num_infections', 'num_uci', 'num_dead', 'Number of infection (%)',
                              'Number of hospitalisation (%)', 'Number of Intensive Care Unit (%)', 'Number of deaths (%)',
                           'ave_7_num_infections'], axis = 1)

In [112]:
hosp['ave_7_num_hosp'] = hosp.iloc[:,0].rolling(window=7).mean()

In [114]:
hosp = hosp.drop(columns = ['num_hosp'], axis = 1)

In [116]:
hosp = hosp[6:]

In [117]:
train, test = hosp[:-7], hosp[-7:]

In [119]:
model=ARMA(train, order = (19, 11)).fit(disp=False)
print('rsme:', mse(model.predict(len(train), len(hosp)-1), test)**0.5, 'daily infections')

rsme: 20.279147528344826 daily infections


In [120]:
hosp = hosp.append(pd.Series(name='2021-06-11 00:00:00'))
hosp = hosp.append(pd.Series(name='2021-06-12 00:00:00'))
hosp = hosp.append(pd.Series(name='2021-06-13 00:00:00'))
hosp = hosp.append(pd.Series(name='2021-06-14 00:00:00'))
hosp = hosp.append(pd.Series(name='2021-06-15 00:00:00'))
hosp = hosp.append(pd.Series(name='2021-06-16 00:00:00'))
hosp = hosp.append(pd.Series(name='2021-06-17 00:00:00'))

In [127]:
train = hosp[:-7]

In [129]:
train

,ave_7_num_hosp
date,
2020-02-23,0.285714
2020-02-24,0.285714
2020-02-25,0.428571
2020-02-26,1.000000
2020-02-27,2.285714
...,...
2021-06-06,195.142857
2021-06-07,189.142857
2021-06-08,182.714286


In [130]:
model = ARMA(train, order = (19, 11)).fit(disp=False)
pred = model.predict(len(train), len(hosp)-1)

In [131]:
pred

2021-06-11    159.873646
2021-06-12    157.179157
2021-06-13    158.815545
2021-06-14    162.838239
2021-06-15    167.128339
2021-06-16    179.841826
2021-06-17    190.712059
Freq: D, dtype: float64

In [134]:
prediction_hosp = pd.DataFrame(pred, columns = ['pred_num_hosp'])

In [135]:
prediction_hosp

,pred_num_hosp
2021-06-11,159.873646
2021-06-12,157.179157
2021-06-13,158.815545
2021-06-14,162.838239
2021-06-15,167.128339
2021-06-16,179.841826
2021-06-17,190.712059


In [136]:
predictions = pd.concat([prediction, prediction_hosp], axis=1)

In [137]:
predictions

,pred_num_infections,pred_num_hosp
2021-06-11,3430.818823,159.873646
2021-06-12,3353.771666,157.179157
2021-06-13,3191.312601,158.815545
2021-06-14,3046.618273,162.838239
2021-06-15,2952.195890,167.128339
2021-06-16,2946.160710,179.841826
2021-06-17,3008.848439,190.712059
2021-06-18,3093.853175,NaN


# UCI

In [138]:
uci = bydate.drop(columns=['num_infections', 'num_hosp', 'num_dead', 'Number of infection (%)',
                              'Number of hospitalisation (%)', 'Number of Intensive Care Unit (%)', 'Number of deaths (%)',
                           'ave_7_num_infections'], axis = 1)

In [140]:
uci['ave_7_num_uci'] = uci.iloc[:,0].rolling(window=7).mean()

In [145]:
uci

,ave_7_num_hosp
date,
2020-01-18,NaN
2020-01-31,NaN
2020-02-09,NaN
2020-02-12,NaN
2020-02-21,NaN
...,...
2021-06-06,20.857143
2021-06-07,17.857143
2021-06-08,16.285714


In [144]:
uci = uci.drop(columns = ['num_uci'], axis = 1)

In [148]:
uci = uci[6:]

In [149]:
train, test = uci[:-7], uci[-7:]

In [150]:
train

,ave_7_num_hosp
date,
2020-02-23,0.000000
2020-02-24,0.142857
2020-02-25,0.142857
2020-02-26,0.285714
2020-02-27,0.428571
...,...
2021-05-30,27.000000
2021-05-31,27.000000
2021-06-01,25.857143


In [151]:
model=ARMA(train, order = (2, 6)).fit(disp=False)
print('rsme:', mse(model.predict(len(train), len(uci)-1), test)**0.5, 'daily uci')

rsme: 6.402380144167087 daily uci


In [152]:
uci = uci.append(pd.Series(name='2021-06-11 00:00:00'))
uci = uci.append(pd.Series(name='2021-06-12 00:00:00'))
uci = uci.append(pd.Series(name='2021-06-13 00:00:00'))
uci = uci.append(pd.Series(name='2021-06-14 00:00:00'))
uci = uci.append(pd.Series(name='2021-06-15 00:00:00'))
uci = uci.append(pd.Series(name='2021-06-16 00:00:00'))
uci = uci.append(pd.Series(name='2021-06-17 00:00:00'))

In [155]:
train = uci[:-7]

In [157]:
model = ARMA(train, order = (2, 6)).fit(disp=False)
pred = model.predict(len(train), len(uci)-1)

In [159]:
prediction_uci = pd.DataFrame(pred, columns = ['pred_num_uci'])

In [160]:
predictions = pd.concat([predictions, prediction_uci], axis=1)

In [161]:
predictions

,pred_num_infections,pred_num_hosp,pred_num_uci
2021-06-11,3430.818823,159.873646,11.825539
2021-06-12,3353.771666,157.179157,9.694381
2021-06-13,3191.312601,158.815545,8.542284
2021-06-14,3046.618273,162.838239,8.306022
2021-06-15,2952.195890,167.128339,8.000496
2021-06-16,2946.160710,179.841826,8.353581
2021-06-17,3008.848439,190.712059,9.258439
2021-06-18,3093.853175,NaN,NaN


# Deaths

In [167]:
deads = bydate.drop(columns=['num_infections', 'num_hosp', 'num_uci', 'Number of infection (%)',
                              'Number of hospitalisation (%)', 'Number of Intensive Care Unit (%)', 'Number of deaths (%)',
                           'ave_7_num_infections'], axis = 1)

In [168]:
deads['ave_7_num_dead'] = deads.iloc[:,0].rolling(window=7).mean()

In [170]:
deads = deads.drop(columns = ['num_dead'], axis = 1)

In [171]:
deads = deads[6:]

In [173]:
train, test = deads[:-7], deads[-7:]

In [175]:
model=ARMA(train, order = (2, 7)).fit(disp=False)
print('rsme:', mse(model.predict(len(train), len(deads)-1), test)**0.5, 'daily deaths')

rsme: 10.137919789778355 daily deaths


In [176]:
deads = deads.append(pd.Series(name='2021-06-11 00:00:00'))
deads = deads.append(pd.Series(name='2021-06-12 00:00:00'))
deads = deads.append(pd.Series(name='2021-06-13 00:00:00'))
deads = deads.append(pd.Series(name='2021-06-14 00:00:00'))
deads = deads.append(pd.Series(name='2021-06-15 00:00:00'))
deads = deads.append(pd.Series(name='2021-06-16 00:00:00'))
deads = deads.append(pd.Series(name='2021-06-17 00:00:00'))

In [178]:
train = deads[:-7]

In [180]:
model = ARMA(train, order = (2, 6)).fit(disp=False)
pred = model.predict(len(train), len(deads)-1)

In [182]:
prediction_dead = pd.DataFrame(pred, columns = ['pred_num_dead'])

In [183]:
predictions = pd.concat([predictions, prediction_dead], axis=1)

In [190]:
predictions = predictions[:7]

In [191]:
predictions

,pred_num_infections,pred_num_hosp,pred_num_uci,pred_num_dead
2021-06-11,3430.818823,159.873646,11.825539,13.016844
2021-06-12,3353.771666,157.179157,9.694381,12.667172
2021-06-13,3191.312601,158.815545,8.542284,13.765634
2021-06-14,3046.618273,162.838239,8.306022,14.791417
2021-06-15,2952.195890,167.128339,8.000496,16.457542
2021-06-16,2946.160710,179.841826,8.353581,18.820792
2021-06-17,3008.848439,190.712059,9.258439,21.751402


In [192]:
predictions.to_csv('predictions.csv', index=True)